In [1]:
import pandas as pd
import numpy as np
import torch as th


In [ ]:
df = pd.read_csv("datasets/Petitions.csv")
df

In [ ]:
df = df["public_petition_text"]
df

In [ ]:
list(df)

In [5]:
data = " ".join(df)

In [ ]:
print(data)

In [103]:
len(set(data.split(" ")))

89517

In [7]:
data = data.lower()

In [8]:
import re

In [9]:
data1 = re.sub('[\\r|\\n|#]+', ' ', data)
data2 = re.sub('[a-zA-Z]+', "", data1)
data3 = re.sub('[0-9]+', "", data2)
data4 = re.sub('[^\s^\w]+', " ", data3)

In [ ]:
print(data4)

In [ ]:
%pip install nltk

In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Fedot\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [ ]:
words = word_tokenize(data4)
words

In [15]:
words=list(set(words))

In [16]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('russian')

In [ ]:
stems = [stemmer.stem(w) for w in words]
stems

In [18]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Fedot\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [ ]:
prepared = [w for w in stems if w not in stopwords]
prepared

In [20]:
len(prepared)

39540

In [27]:
def prepare_data(vocab, words, window):
    dataX = []
    dataY = []
    dct = dict(zip(vocab, range(len(vocab))))
    for i in range(window, len(words)-window):
        st = []
        for j in range(1, window+1):
            st.append(dct[words[i-j]])
            st.append(dct[words[i+j]])
        dataX.append(st)
        dataY.append(dct[words[i]])
    return dataX, dataY

In [28]:
dataX, dataY = prepare_data(["dog", "horse", "cat"], ["dog", "horse", "cat","dog", "horse", "cat","dog", "horse", "cat"], 2)

In [31]:
import torch.nn as nn

In [32]:
class CBoW(nn.Module):
    def __init__(self):
        super(CBoW, self).__init__()
        self.embedder = nn.Embedding(len(words),40)
        self.first_linear = nn.Linear(40, 300)
        self.second_linear = nn.Linear(300, 1)
    def forward(self, x):
        y = self.embedder(x)
        y = self.first_linear(y)
        y = self.second_linear(y)
        return y